In [72]:
%matplotlib qt
import lib.loader as Loader
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import misc
import os
import Evaluation.Evals as evalFunc
from tqdm import tqdm

def reshape_with_zeros(arr,shape):
    ret = np.zeros(shape)
    arr=np.array(arr)
    ret[0:arr.shape[0]*64,0:arr.shape[1]]=np.reshape(arr,(-1,len(arr[0])))
    return ret

def hstack_img_list(arr1,arr2):
    mLen=max(len(arr1),len(arr2))*64
    rs = reshape_with_zeros
    return np.hstack((rs(arr1,(mLen,len(arr1[0]))),rs(arr2,(mLen,len(arr2[0])))))

In [4]:
Cun = Loader.CuneiformSetLoader(path = "/home/jan/Desktop/Cuneiform/img/aia2/CleanSeperated",px=64)

sPath = "/home/jan/Desktop/Cuneiform/img/aia2/Clean/"
if not os.path.exists(sPath):
    os.makedirs(sPath)
data=Cun.dataset[0]

In [11]:
import Nets.SiameseMetric as sNet
netPath = "/home/jan/Desktop/Cuneiform/savedNets/SiameseBackupMetric4_Cun_100.ckpt"
net,saver = sNet.runInit(sNet.backup3Net)
sess = tf.Session()
sNet.runRestore(sess, saver, netPath)

In [23]:
encodings = []
for d in data:
    encodings.append(sess.run(net.enc1,feed_dict={net.x1:d}))

In [28]:
def getSim(encs,ind1,ind2,sess,net):
    e1=np.repeat(encs[ind1],len(encs[ind2]),axis=0)
    e2=np.tile(encs[ind2],(len(encs[ind1]),1))
    sim = sess.run(net.y_pred,feed_dict={net.enc1:e1,net.enc2:e2})
    return sim

In [29]:
s=getSim(encodings,0,1,sess,net)

In [139]:
import time
t=time.time()
s=[]
for i in range(len(data)):
    for j in range(i,len(data)):
        if i!=j:
            s.append([i,j,getSim(encodings,i,j,sess,net)])
            
ti = time.time()-t

In [141]:
import cPickle as pickle

pickle.dump(s,open("simFile.p",'w'))

In [ ]:
lows = []
for i in range(len(s)):
    for j in range(i+1,len(s)):
        if s[i,j]<0.25:
            lows.append((i,j))

In [144]:
similarity[0:10]

[[64, 200, array([[  8.85557979e-02],
         [  3.35016362e-02],
         [  3.33266221e-02],
         [  2.51783460e-01],
         [  1.73821449e-01],
         [  1.73718661e-01],
         [  8.21855813e-02],
         [  1.53197965e-04],
         [  1.16573945e-04],
         [  8.21501389e-02],
         [  1.28992004e-04],
         [  8.32439473e-05]], dtype=float32)], [199, 200, array([[ 0.09549227],
         [ 0.01375187],
         [ 0.0136611 ],
         [ 0.09094663],
         [ 0.00897495],
         [ 0.008946  ],
         [ 0.23134923],
         [ 0.15556858],
         [ 0.15539457]], dtype=float32)], [185, 200, array([[  3.79013449e-01],
         [  3.27070206e-01],
         [  3.27195704e-01],
         [  8.21693167e-02],
         [  1.51280823e-04],
         [  1.02512975e-04],
         [  8.12413543e-02],
         [  1.06739916e-03],
         [  9.72141686e-04],
         [  8.21785033e-02],
         [  1.43814832e-04],
         [  1.12217764e-04],
         [  2.63985116e-0

In [73]:
i=80
plt.imshow(hstack_img_list(data[lows[i][0]],data[lows[i][1]]))
plt.show()

In [104]:
import tkMessageBox
x=[]
plt.ion()

for l in lows:
    l=gui_show(hstack_img_list(data[l[0]],data[l[1]]))
    if l==-1:
        break
    x.append(l)

'Y'

In [115]:
d={}
for x,y in lows:
    if x in d:
        d[x].append(y)
    else:
        d[x]=[y]

In [124]:
delList = []
for key,value in d.iteritems():
    for v in value:
        if v in d:
            d[key].extend(d[v])
            delList.append(v)

In [129]:
x=[]
for key,val in d.iteritems():
    x.append(set(val))
    x[-1].add(key)

[{18, 20},
 {29,
  72,
  83,
  85,
  104,
  106,
  120,
  125,
  127,
  128,
  153,
  162,
  166,
  169,
  173,
  174,
  176,
  179,
  183,
  185,
  194,
  199,
  200,
  202,
  206,
  217,
  232,
  245,
  246},
 {38,
  60,
  61,
  64,
  82,
  83,
  101,
  104,
  106,
  115,
  120,
  125,
  127,
  128,
  153,
  162,
  166,
  169,
  173,
  174,
  176,
  179,
  183,
  185,
  194,
  199,
  200,
  202,
  206,
  217,
  232,
  245,
  246},
 {70, 217},
 {79, 200},
 {91, 200},
 {121, 200}]

In [136]:
num=1
for s in x:
    p=sPath+str(num)
    num+=1
    os.makedirs(p)
    imNum=0
    for val in s:
        for img in data[val]:
            misc.imsave(p+"/"+str(imNum)+".png",img[:,:,0])
            imNum+=1

In [165]:
import matplotlib
def gui_show(data):
    matplotlib.use('TkAgg')
    itL = iter(similarity)
    
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
    # implement the default mpl key bindings
    from matplotlib.backend_bases import key_press_handler
    
    
    from matplotlib.figure import Figure
    
    import sys
    if sys.version_info[0] < 3:
        import Tkinter as Tk
    else:
        import tkinter as Tk
    x=[]
    root = Tk.Tk()
    root.wm_title("Embedding in TK")
    
    
    f = Figure(figsize=(5, 4), dpi=100)
    a = f.add_subplot(111)
    
    v=itL.next()
    a.imshow(hstack_img_list(data[v[0]],data[v[1]]))
    
    
    # a tk.DrawingArea
    canvas = FigureCanvasTkAgg(f, master=root)
    canvas.show()
    canvas.get_tk_widget().pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)
    
    toolbar = NavigationToolbar2TkAgg(canvas, root)
    toolbar.update()
    canvas._tkcanvas.pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)
    
    
    def on_key_event(event):
        print('you pressed %s' % event.key)
        key_press_handler(event, canvas, toolbar)
        
    def _same(*args):
        x.append('Yes')
        v=itL.next()
        a.clear()
        print v[0],v[1]
        a.imshow(hstack_img_list(data[v[0]],data[v[1]]))
        
        canvas.show()
        root.update()
        #_quit()
    
    def _differ(*args):
        x.append('No')
        #_quit()
    canvas.mpl_connect('key_press_event', on_key_event)
    
    def stop():
        x.append(-1)
        _quit()
    
    
    def _quit():
        root.quit()     # stops mainloop
        root.destroy()  # this is necessary on Windows to prevent
                        # Fatal Python Error: PyEval_RestoreThread: NULL tstate
    
    button = Tk.Button(master=root, text='same', command=_same)
    button2 = Tk.Button(master=root, text='different', command=_differ)
    button3 = Tk.Button(master=root, text='stop', command=stop)
    button.pack(side=Tk.BOTTOM)
    button2.pack(side=Tk.BOTTOM)
    button3.pack(side=Tk.BOTTOM)
    
    Tk.mainloop()
    # If you put root.destroy() here, it will cause an error if
    # the window is closed with the window manager.
    return x[0]

In [166]:
gui_show(data)

you pressed left


you pressed left


you pressed left


-1